In [1]:
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import requests
from io import BytesIO

def write_csv_file(filename, json_data):
    df = pd.DataFrame(json_data)
    df.to_csv(filename, index=False)
def rgb_to_hex(r, g, b):
    return '#{:02x}{:02x}{:02x}'.format(r, g, b)
    return '#{:02x}{:02x}{:02x}'.format(tuple[0], tuple[1], tuple[2])
def get_color_palette(image_path):
  # Load the image using mpimg.imread(). Use a raw string (prefix r) or escape the backslashes.
  image = mpimg.imread(image_path)

  # Get the dimensions (width, height, and depth) of the image
  w, h, d = tuple(image.shape)

  # Reshape the image into a 2D array, where each row represents a pixel
  pixel = np.reshape(image, (w * h, d))

  # Import the KMeans class from sklearn.cluster

  # Set the desired number of colors for the image
  n_colors = 18

  # Create a KMeans model with the specified number of clusters and fit it to the pixels
  model = KMeans(n_clusters=n_colors, random_state=42).fit(pixel)

  # Get the cluster centers (representing colors) from the model
  colour_palette = np.uint8(model.cluster_centers_)

  # Display the color palette as an image
  plt.imshow([colour_palette])

  # Show the plot
  plt.show()

In [6]:
def get_color_palette_from_image(image, n_colors=18, showPalette=False):
  image = np.array(image)

  # Get the dimensions (width, height, and depth) of the image
  w, h, d = tuple(image.shape)

  # Reshape the image into a 2D array, where each row represents a pixel
  pixel = np.reshape(image, (w * h, d))

  # Create a KMeans model with the specified number of clusters and fit it to the pixels
  model = KMeans(n_clusters=n_colors, random_state=42).fit(pixel)

  # Get the cluster centers (representing colors) from the model
  colour_palette = np.uint8(model.cluster_centers_)

  if showPalette:
    plt.imshow([colour_palette])
    plt.show()

  return colour_palette

In [11]:
def crop_image_url(url, new_height, new_width):
    try:
        response = requests.get(url)
        im = Image.open(BytesIO(response.content))
        width, height = im.size   

        left = (width - new_width)/2
        top = (height - new_height)/2
        right = (width + new_width)/2
        bottom = (height + new_height)/2

        crop_im = im.crop((left, top, right, bottom)) #Cropping Image 
        # crop_im.save(file_path + "_new.webp") 
        return crop_im
    except:
        # UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x13f0e7060>
        return False

In [2]:
def aclarar_color(hex_color, porcentaje):
    # Convertir el código hexadecimal a valores RGB
    r = int(hex_color[1:3], 16)
    g = int(hex_color[3:5], 16)
    b = int(hex_color[5:7], 16)

    # Aclarar los valores RGB
    r_nuevo = r + (255 - r) * porcentaje
    g_nuevo = g + (255 - g) * porcentaje
    b_nuevo = b + (255 - b) * porcentaje

    # Asegurarse de que los valores están en el rango válido (0-255)
    r_nuevo = min(int(r_nuevo), 255)
    g_nuevo = min(int(g_nuevo), 255)
    b_nuevo = min(int(b_nuevo), 255)

    # Convertir los valores RGB a código hexadecimal
    nuevo_hex = '#{:02x}{:02x}{:02x}'.format(int(r_nuevo), int(g_nuevo), int(b_nuevo))
    return nuevo_hex

In [15]:
def add_color_to_resources(pageResources: str, hex_color):
  if pageResources == '' or pageResources == '[]' or pageResources == [] or pageResources == "['#']":
    return [hex_color]
  # elif "'" in pageResources:
  #   return pageResources.strip("[]").replace("'", "").split(" ")
  elif pageResources:
    print('NOT TRANSFORMED pageResources:', pageResources)
    return []
    # exit(0)

In [ ]:
import csv
storage_photos_url = "https://storage.googleapis.com/catalogo-web/fotos/"
new_width = 300
new_height = 300
porcentaje = 0.7

def update_resources(csv_filepath, limit=-1):
    with open(csv_filepath, mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        line_count = 0

        updated_data = []
        for row in csv_reader:
            if line_count == limit: break
            if line_count == 0:
                line_count += 1

            ''' process '''
            # if productName is empty, then row is empty
            if 'productName' in row and row['productName'] in ['', 'productName', 'Nombre Producto']:
                continue

            # si el SKU es valido, se comienzan a llenar los campos
            if 'SKU' in row and row['SKU'] != '' and row['pageTemplate'] in TEMPLATES_WITH_BG_COLOR:
                url_image = f'{storage_photos_url}{row["SKU"]}-1280.webp'
                new_image = crop_image_url(url_image, new_height, new_width)
                if new_image:
                    color_palette = get_color_palette_from_image(new_image, n_colors=9)

                    r, g, b = color_palette[0][0], color_palette[0][1], color_palette[0][2]
                    hex_color = rgb_to_hex(r, g, b)
                    # lighten_color = adjust_lightness(hex_color, lightness)
                    lighten_color = aclarar_color(hex_color, porcentaje)
                    # print('SKU:', i.replace(storage_photos_url, '').replace('-1280.webp', ''))
                    # print('hex_color:', hex_color)
                    # print('lighten_color:', lighten_color)
                    # print()
                    row["pageResources"] = add_color_to_resources(row["pageResources"], lighten_color)

            line_count += 1
            updated_data.append(row)
            
        print(f'\n\nProcessed {line_count} lines.')
        return updated_data

In [ ]:
new_data = update_resources(
  '/Users/Paty.Lopez/Documents/GitHub/catalogo-web/catalogo-web/src/lib/scripts/Vianney Hogar Resources.csv'
)
